<a href="https://colab.research.google.com/github/VinayakMehta007/EV-Vehicle-Demand-Prediction-Model/blob/main/EV_Vehicle_Charge_Demand_Week_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## EV Adoption Forecasting
As electric vehicle (EV) adoption surges, urban planners need to anticipate infrastructure needs—especially charging stations. Inadequate planning can lead to bottlenecks, impacting user satisfaction and hindering sustainability goals.

**Problem Statement:** Using the electric vehicle dataset (which includes information on EV populations, vehicle types, and possibly historical charging usage), create a model to forecast future EV adoption. For example, predict the number of electric vehicles in upcoming years based on the trends in the data.

**Goal:** Build a regression model that forecasts future EV adoption demand based on historical trends in EV growth, types of vehicles, and regional data.

**Dataset:** This dataset shows the number of vehicles that were registered by Washington State Department of Licensing (DOL) each month. The data is separated by county for passenger vehicles and trucks.

- Date: Counts of registered vehicles are taken on this day (the end of this month). - 2017-01-31
2024-02-29
- County: This is the geographic region of a state that a vehicle's owner is listed to reside within. Vehicles registered in Washington
- State: This is the geographic region of the country associated with the record. These addresses may be located in other
- Vehicle Primary Use: This describes the primary intended use of the vehicle.(Passenger-83%, Truck-17%)
- Battery Electric Vehicles (BEVs): The count of vehicles that are known to be propelled solely by an energy derived from an onboard electric battery.
- Plug-In Hybrid Electric Vehicles (PHEVs): The count of vehicles that are known to be propelled from energy partially sourced from an onboard electric battery
- Electric Vehicle (EV) Total: The sum of Battery Electric Vehicles (BEVs) and Plug-in Hybrid Electric Vehicles (PHEVs).
- Non-Electric Vehicle Total: The count of vehicles that are not electric vehicles.
- Total Vehicles: All powered vehicles registered in the county. This includes electric vehicles.
- Percent Electric Vehicles: Comparison of electric vehicles versus their non-electric counterparts.

**Dataset Link:** https://www.kaggle.com/datasets/sahirmaharajj/electric-vehicle-population-size-2024/data

### Import Required Libraries

### pip install pandas numpy matplotlib seaborn scikit-learn # global env

In [1]:
import joblib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

### Load Dataset

In [2]:
# Load data
df = pd.read_csv("Electric_Vehicle_Population_By_County.csv")

### Explore and Understand the Data

In [3]:
df.head() # top 5 rows

,Date,County,State,Vehicle Primary Use,Battery Electric Vehicles (BEVs),Plug-In Hybrid Electric Vehicles (PHEVs),Electric Vehicle (EV) Total,Non-Electric Vehicle Total,Total Vehicles,Percent Electric Vehicles
0,September 30 2022,Riverside,CA,Passenger,7,0,7,460,467,1.50
1,December 31 2022,Prince William,VA,Passenger,1,2,3,188,191,1.57
2,January 31 2020,Dakota,MN,Passenger,0,1,1,32,33,3.03
3,June 30 2022,Ferry,WA,Truck,0,0,0,"3,575","3,575",0.00
4,July 31 2021,Douglas,CO,Passenger,0,1,1,83,84,1.19


Total 20819 data points and 10 features.

In [4]:
# no of rows and cols
df.shape

(20819, 10)

In [5]:
# Data Types, class and memory alloc
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20819 entries, 0 to 20818
Data columns (total 10 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Date                                      20819 non-null  object 
 1   County                                    20733 non-null  object 
 2   State                                     20733 non-null  object 
 3   Vehicle Primary Use                       20819 non-null  object 
 4   Battery Electric Vehicles (BEVs)          20819 non-null  object 
 5   Plug-In Hybrid Electric Vehicles (PHEVs)  20819 non-null  object 
 6   Electric Vehicle (EV) Total               20819 non-null  object 
 7   Non-Electric Vehicle Total                20819 non-null  object 
 8   Total Vehicles                            20819 non-null  object 
 9   Percent Electric Vehicles                 20819 non-null  float64
dtypes: float64(1), object(9)
memory us

As we could see, only Percent Eletric Vehicles is numeric.

In [6]:
df.isnull().sum()

,0
Date,0
County,86
State,86
Vehicle Primary Use,0
Battery Electric Vehicles (BEVs),0
Plug-In Hybrid Electric Vehicles (PHEVs),0
Electric Vehicle (EV) Total,0
Non-Electric Vehicle Total,0
Total Vehicles,0
Percent Electric Vehicles,0


Missing values in County and State.

**Check if any column contain outliers.**

In [7]:
# Compute Q1 and Q3
Q1 = df['Percent Electric Vehicles'].quantile(0.25)
Q3 = df['Percent Electric Vehicles'].quantile(0.75)
IQR = Q3 - Q1

# Define outlier boundaries
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
print('lower_bound:', lower_bound)
print('upper_bound:', upper_bound)

# Identify outliers
outliers = df[(df['Percent Electric Vehicles'] < lower_bound) | (df['Percent Electric Vehicles'] > upper_bound)]
print("Number of outliers in 'Percent Electric Vehicles':", outliers.shape[0])

lower_bound: -3.5174999999999996
upper_bound: 6.9025
Number of outliers in 'Percent Electric Vehicles': 2476


### Data Preprocessing

Basic Data Cleaning

In [11]:
# Convert date
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df[df['Date'].notnull()]

# Standardize column names (optional, to simplify access)
df.columns = df.columns.str.replace(r'[()]', '', regex=True).str.replace(' ', '_')

# Remove commas and convert number columns to numeric
num_cols = ['Battery_Electric_Vehicles_BEVs', 'Plug-In_Hybrid_Electric_Vehicles_PHEVs',
            'Electric_Vehicle_EV_Total', 'Non-Electric_Vehicle_Total', 'Total_Vehicles']

for col in num_cols:
    df[col] = df[col].astype(str).str.replace(',', '').astype(float)

# Fill missing County and State
df['County'] = df['County'].fillna('Unknown')
df['State'] = df['State'].fillna('Unknown')

# Convert categorical columns
for col in ['County', 'State', 'Vehicle_Primary_Use']:
    df[col] = df[col].astype('category')

# --- FEATURE ENGINEERING ---

df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month


# Confirm remaining nulls
print("Missing after fill:")
print(df[['County', 'State']].isnull().sum())

df.head()

Missing after fill:
County    0
State     0
dtype: int64


,Date,County,State,Vehicle_Primary_Use,Battery_Electric_Vehicles_BEVs,Plug-In_Hybrid_Electric_Vehicles_PHEVs,Electric_Vehicle_EV_Total,Non-Electric_Vehicle_Total,Total_Vehicles,Percent_Electric_Vehicles,Year,Month
0,2022-09-30,Riverside,CA,Passenger,7.0,0.0,7.0,460.0,467.0,1.50,2022,9
1,2022-12-31,Prince William,VA,Passenger,1.0,2.0,3.0,188.0,191.0,1.57,2022,12
2,2020-01-31,Dakota,MN,Passenger,0.0,1.0,1.0,32.0,33.0,3.03,2020,1
3,2022-06-30,Ferry,WA,Truck,0.0,0.0,0.0,3575.0,3575.0,0.00,2022,6
4,2021-07-31,Douglas,CO,Passenger,0.0,1.0,1.0,83.0,84.0,1.19,2021,7


Remove Outliers: Cap the values to the IQR bounds

In [12]:
# --- OUTLIER TREATMENT ---

Q1 = df['Percent_Electric_Vehicles'].quantile(0.25)
Q3 = df['Percent_Electric_Vehicles'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Use clip instead of np.where for better performance
df['Percent_Electric_Vehicles'] = df['Percent_Electric_Vehicles'].clip(lower_bound, upper_bound)

# Final check
print("Remaining outliers:", df[(df['Percent_Electric_Vehicles'] < lower_bound) | (df['Percent_Electric_Vehicles'] > upper_bound)].shape[0])

# Sample view of cleaned data
df.sample(5)

Remaining outliers: 0


,Date,County,State,Vehicle_Primary_Use,Battery_Electric_Vehicles_BEVs,Plug-In_Hybrid_Electric_Vehicles_PHEVs,Electric_Vehicle_EV_Total,Non-Electric_Vehicle_Total,Total_Vehicles,Percent_Electric_Vehicles,Year,Month
16279,2021-12-31,Alameda,CA,Passenger,5.0,0.0,5.0,229.0,234.0,2.14,2021,12
1907,2022-01-31,Onslow,NC,Passenger,0.0,1.0,1.0,175.0,176.0,0.57,2022,1
16331,2018-02-28,Clark,NV,Passenger,2.0,2.0,4.0,748.0,752.0,0.53,2018,2
13038,2018-07-31,San Diego,CA,Passenger,7.0,1.0,8.0,2574.0,2582.0,0.31,2018,7
10295,2017-07-31,Grant,WA,Truck,0.0,0.0,0.0,25270.0,25270.0,0.00,2017,7
